In [ ]:
###数据重采样
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
df = pd.read_excel('C:/Users/LENOVO/Desktop/data/1.xlsx', 'sheet1')
df.index=pd.date_range('00:00:00.005800', periods=600000, freq='0.02U')
print(df)
me=df.resample('20U').agg({'V':'mean'})
print(me)
s=df.resample('20U').agg({'V':'std'})
print(s)
m=df.resample('20U').agg({'V':'max'})
print(m)
M=df.resample('20U').agg({'V':'min'})
print(M)
t=df.resample('20U').agg({'V':['max','min']})
print(t)
n=df.resample('20U').agg({'V':'size'})
print(n)
df.plot()
me.plot()
s.plot()
m.plot()
M.plot()
t.plot()
n.plot()

In [ ]:
###导出保存数据文件
t.to_excel('new.xlsx')
writer = pd.ExcelWriter('new.xlsx')
t.to_excel(writer)
writer.save()

In [ ]:
###ruptures数据分割
import numpy as np
import matplotlib.pylab as plt
import ruptures as rpt
import pandas as pd
df = pd.read_excel('C:/Users/LENOVO/new1.xlsx', 'Sheet1')
data_array = np.array(df)
model = "rbf" 
algo = rpt.BottomUp(model=model).fit(data_array)
my_bkps = algo.predict(n_bkps=22)
rpt.show.display(data_array, my_bkps, figsize=(20, 6))
plt.show()
print(my_bkps)

In [ ]:
###小波
import pywt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data=pd.read_excel( 'F:/data/1.xlsx')
def wavelet(data):
    w = pywt.Wavelet('db4') # 选用Daubechies8小波
    maxlev = pywt.dwt_max_level(len(data), w.dec_len)
    threshold = 0.5 # Threshold for filtering   软硬阈值折衷法参数0.5
    coeffs = pywt.wavedec(data, 'db4', level=maxlev) # 将信号进行小波分解
    for i in range(1, len(coeffs)):
        coeffs[i] = pywt.threshold(coeffs[i], threshold*max(coeffs[i]), 'soft') #软阈值滤波
    datarec = pywt.waverec(coeffs, 'db4') #将信号进行小波重构
    return(datarec)
plt.figure(figsize=(15, 10))
plt.subplot(211)
plt.plot(data,linewidth=0.5)
plt.subplot(212)
plt.plot(datarec,linewidth=0.5)
plt.show()

In [ ]:
##小波
import matplotlib.pyplot as plt
import pywt
import sys
import numpy as np
import pandas as pd
# Data format:
# Raw data should be in a .txt file with two columns, separated by tabs:
#  - The first column should be a time-series index
#  - The second column should contain the data to be filtered

# Get data:
file_object  = open('F:/data/1.txt','r',encoding='UTF-16',errors = 'ignore')
raw_data = file_object.readlines()
index = []
data = []
for i in range(len(raw_data) - 1):
    x, y = raw_data[i].split('\t')
    #x=x.replace('\n','')
    #y=y.replace('\n','')
    X = float(x)
    Y = float(y)
    index.append(X)
    data.append(Y)

#Create wavelet object and define parameters
w = pywt.Wavelet('sym4')
maxlev = pywt.dwt_max_level(len(data), w.dec_len)
# maxlev = 2 # Override if desired
print("maximum level is " + str(maxlev))
threshold = 0.14 # Threshold for filtering

# Decompose into wavelet components, to the level selected:
coeffs = pywt.wavedec(data, 'sym4', level=maxlev)

#cA = pywt.threshold(cA, threshold*max(cA))
plt.figure(figsize=(15, 10))
for i in range(1, len(coeffs)):
    plt.subplot(maxlev, 1, i)
    plt.plot(coeffs[i])
    coeffs[i] = pywt.threshold(coeffs[i], threshold*max(coeffs[i]))
    plt.plot(coeffs[i])


datarec = pywt.waverec(coeffs, 'sym4')


mintime = 1000
maxtime = mintime + 2000

plt.figure(figsize=(15, 10))
plt.subplot(2, 1, 1)
plt.plot(index[mintime:maxtime], data[mintime:maxtime])
plt.xlabel('time (s)')
plt.ylabel('voltage (uV)')
plt.title("Raw signal")
plt.subplot(2, 1, 2)
plt.plot(index[mintime:maxtime], datarec[mintime:maxtime])
plt.xlabel('time (s)')
plt.ylabel('voltage (uV)')
plt.title("De-noised signal using wavelet techniques")

plt.tight_layout()
plt.show()

In [ ]:
####多个文件求均值
import pandas as pd
import os
import xlwt
#获取文件集合
filename = os.listdir('F:/data')
#创建循环，遍历每一个Excel文件
for i in range(len(filename)):
   #读取文件
   df = pd.read_excel('F:/data' + filename[i])
   #创建新的一行数据，每个数据为当前列的平均值
   new=pd.DataFrame({'max':df['max'].mean(),'min':df['min'].mean()},index=[1])
   #将创建的行数据添加到当前表中
   df=df.append(new,ignore_index=True)
   #将处理好的数据导出
   df.to_excel('F:/data/' + filename[i])
print('文件处理成功!')

In [ ]:
####提取多个文件最后一行
import pandas
import os

def read_excel(li):
    df = pandas.read_excel(li, header=0)
    return df.values

if __name__ == '__main__':
    path = r'F:\data\\'
    file_list=[]
    xuhao_list=[]
    max_list=[]
    min_list=[]
    for file_name in os.listdir(path):
        file=path+file_name
        excel_list=read_excel(file)
        file_list.append(file_name)
        xuhao_list.append(float(excel_list[len(excel_list)-1][0]))
        max_list.append(float(excel_list[len(excel_list)-1][1]))
        min_list.append(float(excel_list[len(excel_list)-1][2]))
        print(file_name)
        print(float(excel_list[len(excel_list)-1][0]),float(excel_list[len(excel_list)-1][1]),float(excel_list[len(excel_list)-1][2]))

    df1 = pandas.DataFrame({'文件名': file_list, 'id': xuhao_list,"max":max_list,"min":min_list})

    with pandas.ExcelWriter(r'F:\data\1.xlsx') as writer:
        df1.to_excel(writer, sheet_name='Sheet1', index=False)

In [ ]:
###阈值间改值
import pandas as pd
df = pd.read_excel('E:/demo/TICC/20220516-UV96h-10H/test_22-05-16_1029_009.11.xlsx')
df.loc[(df.iloc[:, 1] > -0.1) & (df.iloc[:, 1] < 0.1)] = 0
df.to_excel('E:/demo/TICC/20220516-UV96h-10H/test_22-05-16_1029_009.11.xlsx', index=False)

or

import openpyxl
workbook = openpyxl.load_workbook('E:/demo/TICC/20220516-UV96h-10H/test_22-05-16_1029_009.1.xlsx')
worksheet = workbook.active
for row in worksheet.iter_rows(min_row=2, min_col=2, max_col=2):
    for cell in row:
        if -0.2 <= cell.value <= 0.2:
            cell.value = 0
workbook.save('E:/demo/TICC/20220516-UV96h-10H/test_22-05-16_1029_009.1.xlsx')

In [ ]:
###批量删除表格第二列
import os
import pandas as pd

folder_path = 'F:/局放数据/1'  # 文件夹路径

# 获取文件夹内的所有Excel文件
excel_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx') or f.endswith('.xls')]

for excel_file in excel_files:
    file_path = os.path.join(folder_path, excel_file)
    
    # 读取Excel文件
    df = pd.read_excel(file_path)
    
    # 删除第二列
    df = df.drop(df.columns[1], axis=1)
    
    # 将第三列数据放到第二列
    #  df.iloc[:, 1] = df.iloc[:, 2]
    
    # 保存修改后的Excel文件
df.to_excel(file_path, index=False)

In [ ]:
###批量提取第二列
import os
import pandas as pd

folder_path = 'F:/局放数据（旧）/1'  # 文件夹路径
output_file = 'output.xlsx'  # 输出文件名

# 创建一个空的DataFrame用于存储所有数据
df_combined = pd.DataFrame()

# 获取文件夹内的所有Excel文件
excel_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx') or f.endswith('.xls')]

for excel_file in excel_files:
    file_path = os.path.join(folder_path, excel_file)

    # 读取Excel文件
    df = pd.read_excel(file_path)

    # 提取第二列数据
    column_data = df.iloc[:, 1]
    
    # 获取文件名作为列名
    column_name = os.path.basename(file_path)

    # 将数据添加到合并DataFrame中
    df_combined[column_name] = column_data

# 保存合并的数据到新Excel文件
df_combined.to_excel(output_file, index=False)